# <center>DP

In [1]:
public class TreeNode {
    int val;
    TreeNode left;
    TreeNode right;
    TreeNode(int x) { val = x; }
}

## <font color='dddd00'>337. House Robber III (midium)</font> [Tree and DP]

The thief has found himself a new place for his thievery again. There is only one entrance to this area, called the "root." Besides the root, each house has one and only one parent house. After a tour, the smart thief realized that "all houses in this place forms a binary tree". It will automatically contact the police if two directly-linked houses were broken into on the same night.

Determine the maximum amount of money the thief can rob tonight without alerting the police.

<font color='dd0000'>取得最大金额的情况为：
    
1. 不选根节点：***opt(root) = opt(root.left) + opt(root.right)***


2. 选根节点：  ***opt(root) = root + opt(left.left) + opt(left.right) + opt(right.left) + opt(right.right)***

根据上述递推式可构造出递归
</font>

In [2]:
// 解法1：递归 重复计算重叠子问题 复杂度高

public int rob(TreeNode root) {
    if(root == null) return 0;
    int a = root.val;
    if(root.left != null) a += rob(root.left.left) + rob(root.left.right);
    if(root.right != null) a += rob(root.right.left) + rob(root.right.right);
    int b = rob(root.left) + rob(root.right);
    return Math.max(a, b);
}

In [4]:
// 解法2：使用动态规划，自底向上，保存每一步的结果，最后推出根节点的值
//        要保存每个结点的结果，所以需要一个HashMap；DP要自底向上，所以需要先遍历树，从叶子结点算起

// 方法1：先先序遍历一遍树，把结点保存在栈中，同时初始化HashMap，再遍历一遍栈，实现自底向上
public int rob21(TreeNode root) {
    Stack<TreeNode> s = new Stack<>();
    Map<TreeNode, Integer> opt = new HashMap<>();
    preorder(root, s, opt);
    opt.put(null, 0);     // 叶子节点计算时会去找null值，所以放入表中

    while(!s.isEmpty()){
        TreeNode cur = s.pop();
        int a = opt.get(cur.left) + opt.get(cur.right);  // 不选
        int b = cur.val;
        if(cur.left != null) b += opt.get(cur.left.left) + opt.get(cur.left.right);
        if(cur.right != null) b += opt.get(cur.right.left) + opt.get(cur.right.right);  // 选
        opt.put(cur, Math.max(a, b));
    }
    return opt.get(root);
}

private void preorder(TreeNode x, Stack<TreeNode> s, Map<TreeNode, Integer> opt){
    if(x == null) return;
    s.push(x);
    opt.put(x, 0);
    preorder(x.left, s, opt);
    preorder(x.right, s, opt);
}


// 方法2：没有必要先遍历树，直接通过递归，在递归过程中把结果逐渐放入哈希表
//        动态规划本身是不需要递归的，因为这是一个树结构，所以递归是为了对树自底向上的查找
public int rob22(TreeNode root){
    Map<TreeNode, Integer> opt = new HashMap<>();
    return helper(root, opt);
}

private int helper(TreeNode root, Map<TreeNode, Integer> opt){
    if(root == null) return 0;
    if(opt.containsKey(root)) return opt.get(root);  // 先序遍历，根结点是最后处理的，当表里已经有root了，说明已经遍历完成，返回它的结果
    
    int a = root.val;
    if(root.left != null) a += helper(root.left.left, opt) + helper(root.left.right, opt);
    if(root.right != null) a += helper(root.right.left, opt) + helper(root.right.right,opt);  // 选
    
    int b = helper(root.left, opt) + helper(root.right, opt);  // 不选
    
    int max = Math.max(a, b);
    opt.put(root, max);      // 在表中记录当前结果
    return max;      // 返回值是每个结点能取到的最大值
}

<font color='dd0000'>从另一个角度考虑，每个结点有选/不选两个状态，每个结点能取的最大值的情况为：

1. 选：两个子节点不能选，那么最大值为 "**不选左子结点时，**左子树能取到的最大值 + **不选右子节点时，**右子节点能取到的最大值 + 自己本身的值"


2. 不选：最大值为 "**选左子节点时，**左子树的最大值 + **选右子节点时，**右子树的最大值"

所以，可以用一个大小为2的数组，分别保存每个节点选或不选两个状态的最大值

这样就不必考虑到孙子节点
</font>

In [8]:
// 解法3：res[0]保存不选该节点时的最大值，res[1]表示选该节点时的最大值

public int rob3(TreeNode root){
    int a = solve(root)[0];
    int b = solve(root)[1];
    return Math.max(a, b);
}

public int[] solve(TreeNode root){
    if(root == null) return new int[2];
    
    int[] res = new int[2];
    
    res[0] = Math.max(solve(root.left)[0], solve(root.left)[1]) + Math.max(solve(root.right)[0], solve(root.right)[1]);
    res[1] = root.val + solve(root.left)[0] + solve(root.right)[0];
    
    return res;
}

## <font color='dddd00'>32. Longest Valid Parentheses (hard)

Given a string containing just the characters ```'('``` and ```')'```, find the length of the longest valid (well-formed) parentheses substring.

In [1]:
// 状态设计：dp[i]表示以第i个字符结尾的最长有效长度
// 有：dp[0]一定为0; 
//     若arr[i]=='(',则dp[i]=0
// 当arr[i]==')'时，如果【i-dp[i-1]-1】位置正好是'('，那么dp[i] = dp[i-1]+2+dp[i-dp[i - 1]-1 - 1];否则，dp[i] = 0
// 最后，最大长度就是dp中最大的数值

public int longestValidParentheses(String s) {
    if(s == null || s.length() == 0)
            return 0;
        
    char[] arr = s.toCharArray();
    int[] dp = new int[s.length()];
    // dp[i]表示以字符arr[i]结尾的最大长度
    dp[0] = 0;
    int res = 0;
    for(int i = 1; i < arr.length; i++){
        if(arr[i] == ')'){
            int pre = i - dp[i - 1] - 1;
            if(pre >= 0 && arr[pre] == '(')
                dp[i] = dp[i - 1] + 2 + ((pre - 1 >= 0) ? dp[pre - 1] : 0);
        }
        res = Math.max(res, dp[i]);
    }
    return res;
}